In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn


In [ ]:
assert False, "Dont reun this cell"
homebuyerDF = pd.read_csv('HomeBuyerInfo.csv')

homebuyerDF["LTV"] = homebuyerDF["LoanAmount"]/homebuyerDF["AppraisedValue"]
homebuyerDF["MonthlyDebt"] = homebuyerDF["CarPayment"] + homebuyerDF["CreditCardPayment"] + homebuyerDF["MonthlyMortgagePayment"]
homebuyerDF["DTI"] = homebuyerDF["MonthlyDebt"]/homebuyerDF["GrossMonthlyIncome"]
homebuyerDF["FEDTI"] = homebuyerDF["MonthlyMortgagePayment"]/homebuyerDF["GrossMonthlyIncome"]

#Add Approval Column
homebuyerDF["Approved"] = ""

"""
A potential buyer should be approved for home purchases iff:
CreditScore >= 640
LTV <= 0.8 is preferred
LTV > .8 and < .95 is approved but high risk (high interest rate)
DTI {
    .43 is hard cap
    .36 is soft cap
}

FEDTI <= .28

"""
#Approval is Y, R-LTV, R-DTI, N for HomebuyerDF
for i, row in homebuyerDF.iterrows():
    if (
        row["CreditScore"] >= 640 and
        row["LTV"] <= .8 and
        row["DTI"] <= .36 and
        row["FEDTI"] <= .28
    ):
        homebuyerDF.at[i, "Approved"] = "Y"
    elif (
        row["CreditScore"] >= 640 and
        row["LTV"] <= .95 and
        row["DTI"] <= .36 and
        row["FEDTI"] <= .28
    ):
        homebuyerDF.at[i, "Approved"] = "R-LTV"
    elif (
        row["CreditScore"] >= 640 and
        row["LTV"] <= .8 and
        row["DTI"] <= .43 and
        row["FEDTI"] <= .28
    ):
        homebuyerDF.at[i, "Approved"] = "R-DTI"
    else:
        homebuyerDF.at[i, "Approved"] = "N"

        
#Save to CSV
homebuyerDF.to_csv("AnalyzedHomeBuyerInfo.csv", index=False)



In [37]:
homebuyerDF = pd.read_csv('AnalyzedHomeBuyerInfo.csv')

In [49]:

approvedDF = homebuyerDF[homebuyerDF["Approved"] == "Y"].drop(columns=["Approved", "ID"])
ltv_ristDF = homebuyerDF[homebuyerDF["Approved"] == "R-LTV"].drop(columns=["Approved", "ID"])
dti_riskDF = homebuyerDF[homebuyerDF["Approved"] == "R-DTI"].drop(columns=["Approved", "ID"])
not_approvedDF = homebuyerDF[homebuyerDF["Approved"] == "N"].drop(columns=["Approved", "ID"])



homebuyerDF["Approved"].value_counts()


Approved
N        7234
R-LTV    1417
Y        1106
R-DTI     243
Name: count, dtype: int64

In [50]:
not_approvedDF.describe()

,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyDebt,DTI,FEDTI
count,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000,7234.000000
mean,5267.845452,352.267902,425.024053,325.698092,372931.011612,61631.793665,311299.217947,1957.205857,643.708737,0.834786,2734.497812,0.612327,0.434592
std,2198.370853,87.108939,43.303740,71.902605,77252.995063,29974.007281,69930.824226,531.095660,97.005868,0.071283,538.380067,0.280146,0.213067
min,2000.000000,200.000000,350.000000,200.000000,230023.000000,11511.550000,164058.280000,758.080000,500.000000,0.710000,1375.090000,0.168739,0.097599
25%,3417.000000,278.000000,387.000000,264.000000,305672.000000,37108.835000,254904.102500,1562.432500,563.000000,0.780000,2328.792500,0.407885,0.286254
50%,4917.000000,353.000000,425.000000,326.000000,378592.500000,58436.420000,310761.840000,1905.055000,625.000000,0.840000,2685.950000,0.532900,0.373815
75%,6899.250000,430.000000,463.000000,387.750000,441475.250000,82196.375000,364964.310000,2333.915000,720.000000,0.900000,3109.210000,0.749535,0.528401
max,9999.000000,499.000000,499.000000,449.000000,499996.000000,144250.640000,473887.550000,3477.220000,849.000000,0.950000,4370.350000,2.042367,1.660079


In [51]:
approvedDF.describe()

,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyDebt,DTI,FEDTI
count,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000,1106.000000
mean,7956.439421,348.716094,419.731465,325.643761,348052.849005,85911.620687,262141.228318,1314.963788,755.914105,0.753752,2083.411347,0.266994,0.167531
std,1322.067047,84.326444,43.483967,73.122854,76438.959157,22457.616371,57425.975146,336.102767,59.296455,0.029323,345.311268,0.049368,0.040249
min,4281.000000,201.000000,350.000000,200.000000,230203.000000,46175.600000,165284.450000,703.010000,640.000000,0.710000,1319.920000,0.145565,0.076698
25%,7008.250000,278.250000,381.000000,260.000000,281926.250000,68579.227500,212634.832500,1057.845000,706.000000,0.730000,1828.737500,0.229729,0.135773
50%,8108.000000,344.000000,417.000000,326.000000,338912.500000,82334.310000,255901.340000,1274.840000,762.000000,0.750000,2045.315000,0.267456,0.168095
75%,9062.000000,421.750000,457.750000,390.000000,409867.000000,100934.637500,309587.617500,1524.555000,809.000000,0.780000,2313.940000,0.305980,0.196518
max,9996.000000,499.000000,499.000000,449.000000,499130.000000,143728.930000,397764.000000,2503.960000,849.000000,0.800000,3306.960000,0.359288,0.274250


In [52]:
ltv_ristDF.describe()

,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyDebt,DTI,FEDTI
count,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000,1417.000000
mean,8141.829217,343.906140,423.651376,326.656316,343054.395907,42987.774912,300066.620995,1496.718207,757.326747,0.874651,2264.275723,0.281867,0.185142
std,1273.791981,86.176817,43.745216,71.492205,75284.462882,17669.913211,67351.842227,379.995567,59.814578,0.042228,387.908501,0.046532,0.040591
min,4462.000000,200.000000,350.000000,200.000000,230021.000000,11876.750000,187327.080000,789.780000,640.000000,0.810000,1456.580000,0.150915,0.081053
25%,7246.000000,264.000000,387.000000,266.000000,274064.000000,28853.820000,242831.440000,1194.500000,708.000000,0.840000,1967.530000,0.247140,0.154893
50%,8304.000000,343.000000,424.000000,328.000000,333800.000000,41259.850000,293111.910000,1458.680000,762.000000,0.870000,2226.960000,0.285895,0.185188
75%,9190.000000,415.000000,461.000000,387.000000,402101.000000,54394.530000,356017.500000,1757.670000,810.000000,0.910000,2523.140000,0.320100,0.215597
max,9999.000000,499.000000,499.000000,449.000000,499516.000000,90965.730000,459867.330000,2744.330000,849.000000,0.950000,3407.330000,0.359933,0.279209


In [53]:
dti_riskDF.describe()

,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyDebt,DTI,FEDTI
count,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000
mean,5542.078189,360.888889,424.308642,332.930041,355479.288066,87074.859588,268404.428477,1377.559506,743.246914,0.755144,2162.757037,0.391581,0.246477
std,984.950830,84.259438,40.363834,72.887319,77679.976850,21497.886258,59345.327456,325.436311,61.797964,0.027141,353.402512,0.018459,0.022530
min,3336.000000,201.000000,350.000000,200.000000,232012.000000,49236.600000,169086.960000,726.840000,640.000000,0.710000,1371.840000,0.360123,0.181670
25%,4827.000000,288.000000,390.500000,268.000000,285223.500000,69094.515000,212540.315000,1133.435000,687.000000,0.730000,1908.890000,0.377481,0.232404
50%,5488.000000,375.000000,425.000000,340.000000,359549.000000,85234.200000,269871.120000,1379.310000,743.000000,0.750000,2144.270000,0.389199,0.251039
75%,6214.000000,423.500000,459.000000,396.000000,417561.500000,104572.995000,316479.490000,1616.755000,799.500000,0.780000,2390.675000,0.404349,0.264554
max,8468.000000,499.000000,499.000000,449.000000,499955.000000,139048.910000,393622.240000,2232.460000,849.000000,0.800000,3177.460000,0.429497,0.279434
